# Model Utilities

> A set of functions that I commonly use for building models

In [ ]:
#| default_exp model_utils

In [ ]:
# | export

from gh_pages_example.utils import *
from gh_pages_example.types import *

import collections
import functools
import itertools
import math
import typing

# import chaospy
import fastcore
import more_itertools
from nbdev.showdoc import *
import nptyping
import numpy as np


## A general method for creating model data

In [ ]:
#| export
def model_builder(saved_args:dict, # a dictionary containing parameters we want to vary or hold fixed
                  exclude_args:list[str]=[], # a list of arguments that should be returned as they are
                  override:bool=False, # whether to build the grid if it is very large
                  drop_args:list[str]=['override', 'exclude_args', 'drop_args'], # a list of arguments to drop from the final result
                 ) -> dict: # A dictionary containing items for the desired models
    """Build models for all combinations of the valules in `saved_args`."""
    axes_args1 = {k: np.array(v)
                  for k,v in saved_args.items()
                  if (isinstance(v, np.ndarray)
                      or (isinstance(v, list)
                          and all(isinstance(el, (float, int)) for el in v)))}
    axes_args2 = {k: np.arange(v["start"], v["stop"], v["step"])
                  for k,v in saved_args.items()
                  if (isinstance(v, dict)
                      and v.keys() == {"start", "stop", "step"})}
    axes_args = {**axes_args1, **axes_args2}
    grid = build_grid_from_axes(list(axes_args.values()),
                                override=override)
    models = {}
    # add grid parameters first
    col_end = dict(zip(axes_args.keys(),
                            np.cumsum([v.shape[-1] if np.ndim(v)==2 else 1
                             for v in axes_args.values()])))
    col_start = {arg: col_end[arg] - (v.shape[-1] if np.ndim(v)==2 else 1)
                  for arg, v in axes_args.items()}
    for i, arg in enumerate(axes_args.keys()):
        if col_start[arg] + 1 == col_end[arg]:
            models[arg] = grid[:, col_start[arg]]
        else:
            models[arg] = grid[:, col_start[arg]:col_end[arg]]
    # add fixed parameters next
    for arg, v in saved_args.items():
        if arg not in (exclude_args
                       + drop_args
                       + list(axes_args.keys())):
            models[arg] = np.array([v for _ in grid])
    # add extra variables
    for arg in exclude_args:
        if (arg in saved_args.keys()
            and arg not in drop_args):
            models[arg] = saved_args[arg]
    return models

In [ ]:
show_doc(model_builder)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/model_utils.py#L27){target="_blank" style="float:right; font-size:smaller"}

### model_builder

>      model_builder (saved_args:dict, exclude_args:list[str]=[],
>                     override:bool=False, drop_args:list[str]=['override',
>                     'exclude_args', 'drop_args'])

Build models for all combinations of the valules in `saved_args`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| saved_args | dict |  | a dictionary containing parameters we want to vary or hold fixed |
| exclude_args | list | [] | a list of arguments that should be returned as they are |
| override | bool | False | whether to build the grid if it is very large |
| drop_args | list | ['override', 'exclude_args', 'drop_args'] | a list of arguments to drop from the final result |
| **Returns** | **dict** |  | **A dictionary containing items for the desired models** |

### Tests for `model_builder`

`model_builder` is able to buid arbitrary grids over data and store them in
a map with their respective column names.

In [ ]:
valid_dtypes = typing.Union[float, list[float], np.ndarray, dict]
def example_model(b:valid_dtypes=4, # benefit: The size of the per round benefit of leading the AI development race, b>0
                c:valid_dtypes=1, # cost: The cost of implementing safety recommendations per round, c>0
                s:valid_dtypes={"start":1, # speed: The speed advantage from choosing to ignore safety recommendations, s>1
                                "stop":5.1,
                                "step":0.1}, 
                p:valid_dtypes={"start":0, # avoid_risk: The probability that unsafe firms avoid an AI disaster, p ∈ [0, 1]
                                "stop":1.02,
                                "step":0.02}, 
                B:valid_dtypes=10**4, # prize: The size of the prize from winning the AI development race, B>>b
                W:valid_dtypes=100, # timeline: The anticipated timeline until the development race has a winner if everyone behaves safely, W ∈ [10, 10**6]
                pfo_l:valid_dtypes=0, # detection_risk_lq: The probability that firms who ignore safety precautions are found out by high quality regulators, pfo_h ∈ [0, 1]
                pfo_h:valid_dtypes=0.5, # detection_risk_hq: The probability that firms who ignore safety precautions are found out by low quality regulators, pfo_h ∈ [0, 1]
                λ:valid_dtypes=0, # disaster_penalty: The penalty levied to regulators in case of a disaster
                r_l:valid_dtypes=0, # profit_lq: profits for low quality regulators before including government incentives, r_l ∈ R
                r_h:valid_dtypes=-1, # profit_hq: profits for high quality regulators before including government incentives, r_h ∈ R
                g:valid_dtypes=1, # government budget allocated to regulators per firm regulated, g > 0
                phi_h:valid_dtypes=0, # regulator_impact: how much do regulators punish unsafe firms they detect, by default detected firms always lose the race.
                phi2_h:valid_dtypes=0, # regulator_impact: how much do regulators punish 2 unsafe firms they detect, by default detected firms always lose the race.
                externality:valid_dtypes=0, # externality: damage caused to society when an AI disaster occurs
                decisiveness:valid_dtypes=1, # How decisive the race is after a regulator gets involved
                incentive_mix:valid_dtypes=1, # Composition of incentives, by default government always pays regulator but rescinds payment if unsafe company discovered
                collective_risk:valid_dtypes=0, # The likelihood that a disaster affects all actors
                β:valid_dtypes=1, # learning_rate: the rate at which players imitate each other
                Z:dict={"S1": 50, "S2": 50}, # population_size: the number of firms and regulators
                strategy_set:list[str]=["HQ-AS", "HQ-AU", "HQ-VS",
                                        "LQ-AS", "LQ-AU", "LQ-VS"], # the set of strategy combinations across all sectors
                exclude_args:list[str]=['Z', 'strategy_set'], # a list of arguments that should be returned as they are
                override:bool=False, # whether to build the grid if it is very large
                drop_args:list[str]=['override', 'exclude_args', 'drop_args'], # a list of arguments to drop from the final result
               ) -> dict: # A dictionary containing items from `ModelTypeRegMarket` and `ModelTypeEGT`
    """Initialise Regulatory Market models for all combinations of the provided
    parameter valules."""
    
    saved_args = locals()
    models = model_builder(saved_args,
                           exclude_args=exclude_args,
                           override=override,
                           drop_args=drop_args)
    return models

In [ ]:
models = example_model()
fastcore.test.test_eq(models['s'].shape, (51*41,))

In [ ]:
models = example_model(s=[0, 1])
fastcore.test.test_eq(np.unique(models['s']), [0, 1])

It is also possible
to provide some data which is quasi monte carlo sampled as a 2D array and have
that be repeated for each combination of the other axes. 

It is up to the user to later split the quasi monte carlo sampled data into
their respective variables.

In [ ]:
# paxis, saxis = chaospy.J(chaospy.TruncNormal(mu=0.2, sigma=1, lower=0, upper=1),
#                          chaospy.TruncNormal(mu=2, sigma=2, lower=1, upper=5)
#                          ).sample(1000, rule="halton").round(4)

In [ ]:
# models =  model_builder(saved_args={"s": np.array([paxis, saxis]).T,
#                                     "g": np.arange(0, 3, 0.5)})
# np.array([paxis, saxis]).T.shape, models['s'].shape, models['g'].shape
# fastcore.test.test_eq(models['s'].shape, (6*1000, 2))

In [ ]:
# models = model_builder(saved_args={"s": saxis,
#                                    "g": np.arange(0, 3, 0.5)})
# fastcore.test.test_eq(models['s'].shape, (6*1000,))

In [ ]:
# models = example_model(s=np.array([paxis, saxis]).T,
#                        p = 0,
#                        g=np.arange(0, 3, 0.5))
# fastcore.test.test_eq(models['s'].shape, (6*1000, 2))
# models['p'] = models['s'][:, 0]
# models['s'] = models['s'][:, 1]
# fastcore.test.test_eq(models['p'].shape, (6*1000,))
# fastcore.test.test_eq(models['s'].shape, (6*1000,))

## Efficiently find profiles for hypergeometric sampling

### `find_unique_allocations`


We want to lexicographically sort our 2D array as we want to sort them so that
those thoses with the highest counts for the lowest strategies (according to
their numerical code) for the earliest subgroups considered (players are split
into subgroups based on their allowed sectors, and subgroups are ordered by
their lowest player index).

We can achieve this if we have strategy counts for each subgroup in a list,
with counts for earlier subgroups first. We then build a list of these lists,
one list for each possible way of achieving the same combined strategy count if
we added each strategy count for each subgroup together. 

So, I test that we can correctly convert a list of list of arrays to our desired
2D array.

We then perform our lexicographic sort and take only the last row of the sorted
array. This is the combination of strategy counts which satisfies the properties
we specified above.

In [ ]:
# | export
def find_unique_allocations(n,  # The number of items to allocate
                            k):  # The number of bins that items can be allocated to
    """Find all combinations with replacement of 'n' of the first `k` integers
    where the sum of the integers is `k`."""
    divider_locations = itertools.combinations(range(n+k-1), k-1)
    allocations = [np.diff(d, append=n+k-1, prepend=-1) - 1
                   for d in divider_locations]
    return allocations


#### Tests for `find_unique_allocations`

In [ ]:
fastcore.test.test_eq(find_unique_allocations(2, 4),
                      [[0, 0, 0, 2],
                       [0, 0, 1, 1],
                       [0, 0, 2, 0],
                       [0, 1, 0, 1],
                       [0, 1, 1, 0],
                       [0, 2, 0, 0],
                       [1, 0, 0, 1],
                       [1, 0, 1, 0],
                       [1, 1, 0, 0],
                       [2, 0, 0, 0], ])

fastcore.test.test_eq(find_unique_allocations(4, 3),
                      [[0, 0, 4],
                       [0, 1, 3],
                       [0, 2, 2],
                       [0, 3, 1],
                       [0, 4, 0],
                       [1, 0, 3],
                       [1, 1, 2],
                       [1, 2, 1],
                       [1, 3, 0],
                       [2, 0, 2],
                       [2, 1, 1],
                       [2, 2, 0],
                       [3, 0, 1],
                       [3, 1, 0],
                       [4, 0, 0], ])


#### Tests for building blocks of `find_unique_counts`

In [ ]:
current_sub_group = [["2", "3"], 2]
count_groups = [[[0, 1, 0, 0], [[1, 0, 0, 0]]],
                [[1, 0, 0, 0], [[0, 1, 0, 0]]],
                ]
categories, n_draws = current_sub_group
n_categories = len(categories)
counts = find_unique_allocations(n_draws, n_categories)
n_all_categories = len(count_groups[0][0])
allocations = np.zeros((len(counts), n_all_categories))
for category, count in zip(categories, np.array(counts).T):
    allocations[:, int(category)] = count

fastcore.test.test_eq(allocations,
                      [[0, 0, 0, 2],
                       [0, 0, 1, 1],
                       [0, 0, 2, 0], ])


current_sub_group = [["1", "3"], 2]
count_groups = [[[0, 1, 0, 0], [[1, 0, 0, 0]]],
                [[1, 0, 0, 0], [[0, 1, 0, 0]]],
                ]
categories, n_draws = current_sub_group
n_categories = len(categories)
counts = find_unique_allocations(n_draws, n_categories)
n_all_categories = len(count_groups[0][0])
allocations = np.zeros((len(counts), n_all_categories))
for category, count in zip(categories, np.array(counts).T):
    allocations[:, int(category)] = count

fastcore.test.test_eq(allocations,
                      [[0, 0, 0, 2],
                       [0, 1, 0, 1],
                       [0, 2, 0, 0], ])


In [ ]:
fastcore.test.test_eq(np.sum([[1, 1], np.array([1, 2])], axis=1),
                      [2, 3])
fastcore.test.test_eq(":".join(map(str, map(int, [1, 2, 3.0]))),
                      "1:2:3")


In [ ]:
repeated_counts = [[np.array([1, 2, 3]),
                    np.array([2, 3, 4]), ],
                   [np.array([2, 3, 4]),
                    np.array([1, 2, 3]), ]]
y = np.vstack([np.hstack(seq) for seq in repeated_counts])
fastcore.test.test_eq(y,
                      np.array([[1, 2, 3, 2, 3, 4],
                                [2, 3, 4, 1, 2, 3]]))
fastcore.test.test_eq(np.lexsort(y.T[::-1]),
                      [0, 1])
fastcore.test.test_eq(np.lexsort(y.T[::-1])[-1],
                      1)
fastcore.test.test_eq(y[np.lexsort(y.T[::-1])[-1]],
                      [2, 3, 4, 1, 2, 3])
fastcore.test.test_eq(np.split(y[np.lexsort(y.T[::-1])[-1]], 2),
                      [[2, 3, 4], [1, 2, 3]])


### `find_unique_counts`

In [ ]:
# | export
def find_unique_counts(count_groups, current_sub_group):
    """Build a list of all unique count groups given an allocation and a list
    of existing unique counts."""
    new_count_groups = []
    combined_counts = collections.defaultdict(list)
    # First, we turn our new allocation rules into an array of all possible
    # allocations for this sub-group.
    # Each player in a sub-group follows the same allocation rules so they can
    # each have any relevant strategy.
    # Each allocation must have all relevant strategy counts recorded, including
    # those which are 0.
    categories, n_draws = current_sub_group
    n_categories = len(categories)
    counts = find_unique_allocations(n_draws, n_categories)
    n_all_categories = len(count_groups[0][0])
    allocations = np.zeros((len(counts), n_all_categories))
    for category, count in zip(categories, np.array(counts).T):
        allocations[:, int(category)] = count
    for group in count_groups:
        for allocation in allocations:
            count_seq = [*group, [*map(int, allocation)]]
            combined_count = np.sum(count_seq, axis=0)
            count_key = ":".join(map(str, map(int, combined_count)))
            combined_counts[count_key].append(count_seq)
    for repeated_counts in combined_counts.values():
        if len(repeated_counts) == 1:
            new_count_groups.append(repeated_counts[0])
        else:
            n_sub_groups = len(repeated_counts[0])
            # Convert the repeated counts into a 2D array
            x = np.vstack([np.hstack(seq) for seq in repeated_counts])
            # Lexicographic sort means that lower strategies most common on bottom
            # row (for earlier sub-groups).
            candidate = x[np.lexsort(x.T[::-1])[-1]]
            # Split row back into a list of counts for each sub-group
            # Throw an error if for some reason the sub-groups have differently
            # sized arrays (only possible if wrong values passed to this function).
            candidate = np.split(candidate, n_sub_groups)
            new_count_groups.append(candidate)
    return new_count_groups


#### Tests for `find_unique_counts`

In [ ]:
current_sub_group = [["2", "3"], 2]
count_groups = [[[0, 0, 0, 0]], ]
result = find_unique_counts(count_groups, current_sub_group)
expected = [[[0, 0, 0, 0], [0, 0, 0, 2]],
            [[0, 0, 0, 0], [0, 0, 1, 1]],
            [[0, 0, 0, 0], [0, 0, 2, 0]]]


In [ ]:
current_sub_group = [["2", "3"], 2]
count_groups = [[[1, 0, 0, 0], [0, 1, 0, 0]], ]
result = find_unique_counts(count_groups, current_sub_group)
expected = [[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 2]],
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 1]],
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, 0]]]

current_sub_group = [["2", "3"], 2]
# Notice that the given count groups are redundant since if we had created them
# using `find_unique_counts`, then we would have only kept the bottom one.
count_groups = [[[0, 1, 0, 0], [1, 0, 0, 0]],
                [[1, 0, 0, 0], [0, 1, 0, 0]],
                ]
result = find_unique_counts(count_groups, current_sub_group)
expected = [[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 2]],
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 1]],
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, 0]]]


In [ ]:
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1", "S2"], }
sector_strategies = {"S1": ["1", "2"],
                     "S2": ["3", "4"],
                     "S3": ["5", "6"]}

rules = collections.defaultdict(list)
for player, sectors in allowed_sectors.items():
    available_strategies = []
    for sector in sectors:
        available_strategies += sector_strategies[sector]
    available_strategies = np.unique(available_strategies)
    rule = ":".join(available_strategies)
    rules[rule].append(player)

subgroups = [[str.split(rule, ':'), len(players)]
             for rule, players in rules.items()]

fastcore.test.test_eq(subgroups,
                      [[['1', '2'], 1], [['1', '2', '3', '4'], 1]])

In [ ]:
sorted(["P10", "P1","P2"])
player_strings = ["P10", "P1","P2"]
player_numbers = [int("".join(string[1:]))
                  for string in player_strings]
players = [f"P{x}" for x in sorted(player_numbers)]
fastcore.test.test_eq(players, ["P1", "P2","P10"])

## Methods for creating strategy profile codes for a set of models.

I also need a method for creating all possible player profiles. I need to use this method before the profile filters can do any filtering.

We can also apply our profile filters to this list to know exactly which profiles we need to write payoffs for.

I also use this method to validate that we have passed sufficient information to payoffs, and to warn us about what is missing.

Note that it creates strategy profiles where some players may not be present
(as represented by the strategy `0`). It is useful to think of `0` as
representing a "null sector" as it captures the possibility that a player
isn't sampled from any of the sectors. 

If one instead wants to capture the possibility that particular sectors may be less likely to participate, they may achieve this by creating a new strategy for that sector to
represent a member from that sector as not participating in the interaction. It is also
possible to achieve a similar effect by using `sector_weights`.

I prefer to reserve `0` as a code for a missing player, so I index strategies from `1`.
However, note that if you wish to index your strategies from `0` instead, the
profile filters are flexible enough to allow you to do so without issue. If you
wish to later add the possibility of missing players, you just need to assign it
a new code which you use to indicate the relevant payoffs.

In [ ]:
#| export
@multi
def create_profiles(models):
    """Create strategy profiles relevant to the strategies, players, and sectors
    in models according to the given rule"""
    return models.get('profiles_rule')

@method(create_profiles, "anonymous")
def create_profiles(models):
    """Create strategy profiles for each combination of strategies. As
    the payoffs do not care who each player is (i.e. they are anonymous), we
    only create one profile for each unique strategy count. In this case we
    always keep alike strategies together with the lowest strategy on the right,
    weakly increasing as we read left, where possible. 

    When some players only
    draw from a subset of sectors, we follow a similar process for each unique
    subset of sectors that groups of players are allowed to sample from.

    Follow this routine: given one subset and the previous result of this
    routine, find all possible strategy counts using the previous set of
    strategy counts and the allowed player allocations for this subset. When
    we repeat a strategy count, keep only the strategy count (for the subset)
    associated with the previous strategy count with the highest value of
    lower strategies. This way, we have a reducer function that returns a
    list of lists of strategy count components which always add up to a unique
    strategy count.
    """

    sector_strategies = models.get('sector_strategies', {})
    allowed_sectors = models.get('allowed_sectors', {})
    n_strategies = [len(v) for v in sector_strategies.values()]
    zero_strategy = False
    for strategies in sector_strategies.values():
        if "0" in strategies:
            zero_strategy = True
    n_strategies_total = np.sum(n_strategies) + (1 - zero_strategy)

    rules = collections.defaultdict(list)
    for player, sectors in allowed_sectors.items():
        available_strategies = []
        for sector in sectors:
            available_strategies += sector_strategies[sector]
        available_strategies = np.unique(available_strategies)
        rule = ":".join(available_strategies)
        rules[rule].append(player)

    subgroups = [[str.split(rule, ':'), len(players)]
                 for rule, players in rules.items()]
    zero_counts = [[np.zeros((n_strategies_total), dtype=int)]]
    unique_counts = functools.reduce(find_unique_counts,
                                     subgroups,
                                     zero_counts)
    profiles = []
    for unique_count in unique_counts:
        player_assignments = {}
        unique_count = unique_count[1:]
        for i, subgroup_count in enumerate(unique_count):
            subgroup_key = ":".join(map(str, subgroups[i][0]))
            player_strings = rules[subgroup_key]
            player_numbers = [int("".join(string[1:]))
                              for string in player_strings]
            subgroup_players = [f"P{x}" for x in sorted(player_numbers)]
            count = subgroup_count.copy()
            for player in subgroup_players:
                strategy = np.argmax(np.array(count) > 0)
                player_assignments[player] = strategy
                count[strategy] -= 1
        profile = [player_assignments[f"P{i+1}"]
                   for i, _ in enumerate(player_assignments)]
        profile = "-".join(map(str, profile[::-1]))
        profiles.append(profile)
    return {**models, "profiles": profiles}

@method(create_profiles, "from_strategy_count")
def create_profiles(models):
    """Create all valid profiles given allowed_actions, where the count of each
    strategy is equal to the given strategy_count."""
    strategy_count = models['strategy_count']
    allowed_actions = models.get('allowed_actions', {}) # Optional
    # If provided, allowed_actions must provide rules for every player.
    assert (len(allowed_actions) == sum(strategy_count.values())
            or allowed_actions == {})
    # Create a set based on the strategy_count
    strategy_pool = [[strategy]*count for strategy, count in strategy_count.items()]
    strategy_pool = [val for sublist in strategy_pool for val in sublist]
    profiles = []
    if allowed_actions == {}:
        for profile in more_itertools.distinct_permutations(strategy_pool,
                                                            r=len(strategy_pool)):
            profile = "-".join(map(str, profile))
            profiles.append(profile)
    else:
        for profile in more_itertools.distinct_permutations(strategy_pool,
                                                            r=len(strategy_pool)):
            profile_valid = True
            for player, strategy in enumerate(string_to_tuple(profile)):
                if strategy not in allowed_actions[f"P{player + 1}"]:
                    profile_valid = False
            if profile_valid:
                profiles.append(profile)
    return {**models, "profiles": profiles}

# @method(create_profiles)
# def create_profiles(models):
#     """Create all strategy profiles for the set of models."""
#     sector_strategies = models.get('sector_strategies', {})
#     allowed_sectors = models.get('allowed_sectors', {})
#     n_players = models.get('n_players', len(allowed_sectors))
#     n_strategies = models.get('n_strategies',
#                               [len(v) for v in sector_strategies.values()])
#     n_strategies_total = np.sum(n_strategies)
#     zero_strategy = False
#     for strategies in sector_strategies.values():
#         if "0" in strategies:
#             zero_strategy = True
#     n_strategies_total += (1 - zero_strategy)  # Add null sector if not present
#     n_profiles = n_strategies_total ** n_players
#     strategy_axis = np.arange(n_strategies_total)[:, None]
#     grid = build_grid_from_axes([strategy_axis for _ in range(n_players)])
#     profiles = []
#     for row in grid:
#         profile = "-".join(map(str, row))
#         profiles.append(profile)
#     fastcore.test.test_eq(len(profiles), n_profiles)
#     return {**models, "profiles": profiles}


@method(create_profiles)
def create_profiles(models):
    """Create all strategy profiles for the set of models."""
    sector_strategies = models.get('sector_strategies', {})
    allowed_sectors = models.get('allowed_sectors', {})
    n_players = models.get('n_players', len(allowed_sectors))
    n_strategies = models.get('n_strategies',
                              [len(v) for v in sector_strategies.values()])
    n_strategies_total = np.sum(n_strategies)
    if sector_strategies=={}:
        strategies = np.arange(n_strategies_total)
    else:
        strategies = np.unique([strategy
                                for sector in sector_strategies.keys()
                                for strategy in sector_strategies[sector]])
    n_profiles = n_strategies_total ** n_players
    strategy_axis = strategies[:, None]
    grid = build_grid_from_axes([strategy_axis for _ in range(n_players)])
    profiles = []
    for row in grid:
        profile = "-".join(map(str, row))
        profiles.append(profile)
    fastcore.test.test_eq(len(profiles), n_profiles)
    return {**models, "profiles": profiles}

##### Tests for `create_profiles`

Here are tests for the default method, where all possible profiles are created
for a given number of strategies and players.

In [ ]:
result = create_profiles({"n_players": 2, "n_strategies": 3})
fastcore.test.test_eq(result['profiles'],
                      ['0-0', '0-1', '0-2',
                       '1-0', '1-1', '1-2',
                       '2-0', '2-1', '2-2'])

result = create_profiles({"n_players": 2, "n_strategies": [3, 1]})
fastcore.test.test_eq(result['profiles'],
                      ["0-0", "0-1", "0-2", "0-3",
                       "1-0", "1-1", "1-2", "1-3",
                       "2-0", "2-1", "2-2", "2-3",
                       "3-0", "3-1", "3-2", "3-3", ])

fastcore.test.test_eq(create_profiles({"n_players": 2,
                                       "n_strategies": [2, 2]})['profiles'],
                      create_profiles({"n_players": 2,
                                       "n_strategies": 4})['profiles'])

models = {"allowed_sectors": {"P1": ["S1"],
                              "P2": ["S2"]},
          "sector_strategies": {"S1": [0, 1],
                                "S2": [2, 3]}}
fastcore.test.test_eq(create_profiles(models)['profiles'],
                      create_profiles({"n_players": 2,
                                       "n_strategies": 4})['profiles'])


I'll test the `"from_strategy_count"` method. Notice that it grows
quickly for larger strategy counts with more choices. As a rule of thumb, this
will become untenable with more than 10 players, especially when there are more
choices to choose between.

In [ ]:
strategy_count = {"1": 2, "2": 2}
models = {"strategy_count": strategy_count,
          "profiles_rule": "from_strategy_count"}
results = create_profiles(models)
length = (math.factorial(sum(strategy_count.values()))
          / math.prod([math.factorial(v) for v in strategy_count.values()]))
fastcore.test.test_eq(length, len(results['profiles']))
expected = ["1-1-2-2",
            "1-2-1-2",
            "1-2-2-1",
            "2-1-1-2",
            "2-1-2-1",
            "2-2-1-1",]
fastcore.test.test_eq(expected, results['profiles'])

strategy_count = {"1": 2, "2": 2, "3": 4, "4": 3}
models = {"strategy_count": strategy_count,
          "profiles_rule": "from_strategy_count"}
results = create_profiles(models)
length = (math.factorial(sum(strategy_count.values()))
          / math.prod([math.factorial(v) for v in strategy_count.values()]))
fastcore.test.test_eq(length, len(results['profiles']))

We also include tests for the `create_profiles` "anonymous" method. which only
creates one profile for each unique count of strategies.

In [ ]:
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1", "S2"], }
# Third sector irrelevant given allowed sectors
sector_strategies = {"S1": ["1", "2"],
                     "S2": ["3", "4"],
                     "S3": ["5", "6"]}
models = {"allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "profiles_rule": "anonymous", }
profiles = create_profiles(models)['profiles']
expected = ['4-2', '3-2', '2-2', '2-1', '4-1', '3-1', '1-1']
fastcore.test.test_eq(profiles, expected)
fastcore.test.test_eq(sorted(profiles),
                      ['1-1', '2-1', '2-2', '3-1', '3-2', '4-1', '4-2'])


In [ ]:
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1", "S2"], }
# Third sector irrelevant given allowed sectors
sector_strategies = {"S1": ["1", "0"],
                     "S2": ["3", "4"],
                     "S3": ["5", "6"]}
models = {"allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "profiles_rule": "anonymous", }
profiles = create_profiles(models)['profiles']
expected = ['4-1', '3-1', '1-1', '1-0', '4-0', '3-0', '0-0']
fastcore.test.test_eq(profiles, expected)
fastcore.test.test_eq(sorted(profiles),
                      ['0-0', '1-0', '1-1', '3-0', '3-1', '4-0', '4-1'])

In [ ]:
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1", "S2"],
                   "P3": ["S1", "S2"],
                   "P4": ["S1", "S2"],
                   "P5": ["S1", "S2"],
                   "P6": ["S1", "S2"],
                   "P7": ["S1", "S2"],
                   "P8": ["S1", "S2"],
                   "P9": ["S1", "S2"],
                   "P10": ["S1", "S2"], }
# Third sector irrelevant given allowed sectors
sector_strategies = {"S1": ["1", "2"],
                     "S2": ["3", "4"],
                     "S3": ["5", "6"]}
models = {"allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "profiles_rule": "anonymous", }
profiles = create_profiles(models)['profiles']
expected = ['1-1-1-1-1-1-1-1-1-1',
            '2-1-1-1-1-1-1-1-1-1',
            '2-2-1-1-1-1-1-1-1-1',
            '2-2-2-1-1-1-1-1-1-1',
            '2-2-2-2-1-1-1-1-1-1',
            '2-2-2-2-2-1-1-1-1-1',
            '2-2-2-2-2-2-1-1-1-1',
            '2-2-2-2-2-2-2-1-1-1',
            '2-2-2-2-2-2-2-2-1-1',
            '2-2-2-2-2-2-2-2-2-1',
            '2-2-2-2-2-2-2-2-2-2',
            '3-1-1-1-1-1-1-1-1-1',
            '3-2-1-1-1-1-1-1-1-1',
            '3-2-2-1-1-1-1-1-1-1',
            '3-2-2-2-1-1-1-1-1-1',
            '3-2-2-2-2-1-1-1-1-1',
            '3-2-2-2-2-2-1-1-1-1',
            '3-2-2-2-2-2-2-1-1-1',
            '3-2-2-2-2-2-2-2-1-1',
            '3-2-2-2-2-2-2-2-2-1',
            '3-2-2-2-2-2-2-2-2-2',
            '3-3-1-1-1-1-1-1-1-1',
            '3-3-2-1-1-1-1-1-1-1',
            '3-3-2-2-1-1-1-1-1-1',
            '3-3-2-2-2-1-1-1-1-1',
            '3-3-2-2-2-2-1-1-1-1',
            '3-3-2-2-2-2-2-1-1-1',
            '3-3-2-2-2-2-2-2-1-1',
            '3-3-2-2-2-2-2-2-2-1',
            '3-3-2-2-2-2-2-2-2-2',
            '3-3-3-1-1-1-1-1-1-1',
            '3-3-3-2-1-1-1-1-1-1',
            '3-3-3-2-2-1-1-1-1-1',
            '3-3-3-2-2-2-1-1-1-1',
            '3-3-3-2-2-2-2-1-1-1',
            '3-3-3-2-2-2-2-2-1-1',
            '3-3-3-2-2-2-2-2-2-1',
            '3-3-3-2-2-2-2-2-2-2',
            '3-3-3-3-1-1-1-1-1-1',
            '3-3-3-3-2-1-1-1-1-1',
            '3-3-3-3-2-2-1-1-1-1',
            '3-3-3-3-2-2-2-1-1-1',
            '3-3-3-3-2-2-2-2-1-1',
            '3-3-3-3-2-2-2-2-2-1',
            '3-3-3-3-2-2-2-2-2-2',
            '3-3-3-3-3-1-1-1-1-1',
            '3-3-3-3-3-2-1-1-1-1',
            '3-3-3-3-3-2-2-1-1-1',
            '3-3-3-3-3-2-2-2-1-1',
            '3-3-3-3-3-2-2-2-2-1',
            '3-3-3-3-3-2-2-2-2-2',
            '3-3-3-3-3-3-1-1-1-1',
            '3-3-3-3-3-3-2-1-1-1',
            '3-3-3-3-3-3-2-2-1-1',
            '3-3-3-3-3-3-2-2-2-1',
            '3-3-3-3-3-3-2-2-2-2',
            '3-3-3-3-3-3-3-1-1-1',
            '3-3-3-3-3-3-3-2-1-1',
            '3-3-3-3-3-3-3-2-2-1',
            '3-3-3-3-3-3-3-2-2-2',
            '3-3-3-3-3-3-3-3-1-1',
            '3-3-3-3-3-3-3-3-2-1',
            '3-3-3-3-3-3-3-3-2-2',
            '3-3-3-3-3-3-3-3-3-1',
            '3-3-3-3-3-3-3-3-3-2',
            '4-1-1-1-1-1-1-1-1-1',
            '4-2-1-1-1-1-1-1-1-1',
            '4-2-2-1-1-1-1-1-1-1',
            '4-2-2-2-1-1-1-1-1-1',
            '4-2-2-2-2-1-1-1-1-1',
            '4-2-2-2-2-2-1-1-1-1',
            '4-2-2-2-2-2-2-1-1-1',
            '4-2-2-2-2-2-2-2-1-1',
            '4-2-2-2-2-2-2-2-2-1',
            '4-2-2-2-2-2-2-2-2-2',
            '4-3-1-1-1-1-1-1-1-1',
            '4-3-2-1-1-1-1-1-1-1',
            '4-3-2-2-1-1-1-1-1-1',
            '4-3-2-2-2-1-1-1-1-1',
            '4-3-2-2-2-2-1-1-1-1',
            '4-3-2-2-2-2-2-1-1-1',
            '4-3-2-2-2-2-2-2-1-1',
            '4-3-2-2-2-2-2-2-2-1',
            '4-3-2-2-2-2-2-2-2-2',
            '4-3-3-1-1-1-1-1-1-1',
            '4-3-3-2-1-1-1-1-1-1',
            '4-3-3-2-2-1-1-1-1-1',
            '4-3-3-2-2-2-1-1-1-1',
            '4-3-3-2-2-2-2-1-1-1',
            '4-3-3-2-2-2-2-2-1-1',
            '4-3-3-2-2-2-2-2-2-1',
            '4-3-3-2-2-2-2-2-2-2',
            '4-3-3-3-1-1-1-1-1-1',
            '4-3-3-3-2-1-1-1-1-1',
            '4-3-3-3-2-2-1-1-1-1',
            '4-3-3-3-2-2-2-1-1-1',
            '4-3-3-3-2-2-2-2-1-1',
            '4-3-3-3-2-2-2-2-2-1',
            '4-3-3-3-2-2-2-2-2-2',
            '4-3-3-3-3-1-1-1-1-1',
            '4-3-3-3-3-2-1-1-1-1',
            '4-3-3-3-3-2-2-1-1-1',
            '4-3-3-3-3-2-2-2-1-1',
            '4-3-3-3-3-2-2-2-2-1',
            '4-3-3-3-3-2-2-2-2-2',
            '4-3-3-3-3-3-1-1-1-1',
            '4-3-3-3-3-3-2-1-1-1',
            '4-3-3-3-3-3-2-2-1-1',
            '4-3-3-3-3-3-2-2-2-1',
            '4-3-3-3-3-3-2-2-2-2',
            '4-3-3-3-3-3-3-1-1-1',
            '4-3-3-3-3-3-3-2-1-1',
            '4-3-3-3-3-3-3-2-2-1',
            '4-3-3-3-3-3-3-2-2-2',
            '4-3-3-3-3-3-3-3-1-1',
            '4-3-3-3-3-3-3-3-2-1',
            '4-3-3-3-3-3-3-3-2-2',
            '4-3-3-3-3-3-3-3-3-1',
            '4-3-3-3-3-3-3-3-3-2',
            '4-4-1-1-1-1-1-1-1-1',
            '4-4-2-1-1-1-1-1-1-1',
            '4-4-2-2-1-1-1-1-1-1',
            '4-4-2-2-2-1-1-1-1-1',
            '4-4-2-2-2-2-1-1-1-1',
            '4-4-2-2-2-2-2-1-1-1',
            '4-4-2-2-2-2-2-2-1-1',
            '4-4-2-2-2-2-2-2-2-1',
            '4-4-2-2-2-2-2-2-2-2',
            '4-4-3-1-1-1-1-1-1-1',
            '4-4-3-2-1-1-1-1-1-1',
            '4-4-3-2-2-1-1-1-1-1',
            '4-4-3-2-2-2-1-1-1-1',
            '4-4-3-2-2-2-2-1-1-1',
            '4-4-3-2-2-2-2-2-1-1',
            '4-4-3-2-2-2-2-2-2-1',
            '4-4-3-2-2-2-2-2-2-2',
            '4-4-3-3-1-1-1-1-1-1',
            '4-4-3-3-2-1-1-1-1-1',
            '4-4-3-3-2-2-1-1-1-1',
            '4-4-3-3-2-2-2-1-1-1',
            '4-4-3-3-2-2-2-2-1-1',
            '4-4-3-3-2-2-2-2-2-1',
            '4-4-3-3-2-2-2-2-2-2',
            '4-4-3-3-3-1-1-1-1-1',
            '4-4-3-3-3-2-1-1-1-1',
            '4-4-3-3-3-2-2-1-1-1',
            '4-4-3-3-3-2-2-2-1-1',
            '4-4-3-3-3-2-2-2-2-1',
            '4-4-3-3-3-2-2-2-2-2',
            '4-4-3-3-3-3-1-1-1-1',
            '4-4-3-3-3-3-2-1-1-1',
            '4-4-3-3-3-3-2-2-1-1',
            '4-4-3-3-3-3-2-2-2-1',
            '4-4-3-3-3-3-2-2-2-2',
            '4-4-3-3-3-3-3-1-1-1',
            '4-4-3-3-3-3-3-2-1-1',
            '4-4-3-3-3-3-3-2-2-1',
            '4-4-3-3-3-3-3-2-2-2',
            '4-4-3-3-3-3-3-3-1-1',
            '4-4-3-3-3-3-3-3-2-1',
            '4-4-3-3-3-3-3-3-2-2',
            '4-4-3-3-3-3-3-3-3-1',
            '4-4-3-3-3-3-3-3-3-2',
            '4-4-4-1-1-1-1-1-1-1',
            '4-4-4-2-1-1-1-1-1-1',
            '4-4-4-2-2-1-1-1-1-1',
            '4-4-4-2-2-2-1-1-1-1',
            '4-4-4-2-2-2-2-1-1-1',
            '4-4-4-2-2-2-2-2-1-1',
            '4-4-4-2-2-2-2-2-2-1',
            '4-4-4-2-2-2-2-2-2-2',
            '4-4-4-3-1-1-1-1-1-1',
            '4-4-4-3-2-1-1-1-1-1',
            '4-4-4-3-2-2-1-1-1-1',
            '4-4-4-3-2-2-2-1-1-1',
            '4-4-4-3-2-2-2-2-1-1',
            '4-4-4-3-2-2-2-2-2-1',
            '4-4-4-3-2-2-2-2-2-2',
            '4-4-4-3-3-1-1-1-1-1',
            '4-4-4-3-3-2-1-1-1-1',
            '4-4-4-3-3-2-2-1-1-1',
            '4-4-4-3-3-2-2-2-1-1',
            '4-4-4-3-3-2-2-2-2-1',
            '4-4-4-3-3-2-2-2-2-2',
            '4-4-4-3-3-3-1-1-1-1',
            '4-4-4-3-3-3-2-1-1-1',
            '4-4-4-3-3-3-2-2-1-1',
            '4-4-4-3-3-3-2-2-2-1',
            '4-4-4-3-3-3-2-2-2-2',
            '4-4-4-3-3-3-3-1-1-1',
            '4-4-4-3-3-3-3-2-1-1',
            '4-4-4-3-3-3-3-2-2-1',
            '4-4-4-3-3-3-3-2-2-2',
            '4-4-4-3-3-3-3-3-1-1',
            '4-4-4-3-3-3-3-3-2-1',
            '4-4-4-3-3-3-3-3-2-2',
            '4-4-4-3-3-3-3-3-3-1',
            '4-4-4-3-3-3-3-3-3-2',
            '4-4-4-4-1-1-1-1-1-1',
            '4-4-4-4-2-1-1-1-1-1',
            '4-4-4-4-2-2-1-1-1-1',
            '4-4-4-4-2-2-2-1-1-1',
            '4-4-4-4-2-2-2-2-1-1',
            '4-4-4-4-2-2-2-2-2-1',
            '4-4-4-4-2-2-2-2-2-2',
            '4-4-4-4-3-1-1-1-1-1',
            '4-4-4-4-3-2-1-1-1-1',
            '4-4-4-4-3-2-2-1-1-1',
            '4-4-4-4-3-2-2-2-1-1',
            '4-4-4-4-3-2-2-2-2-1',
            '4-4-4-4-3-2-2-2-2-2',
            '4-4-4-4-3-3-1-1-1-1',
            '4-4-4-4-3-3-2-1-1-1',
            '4-4-4-4-3-3-2-2-1-1',
            '4-4-4-4-3-3-2-2-2-1',
            '4-4-4-4-3-3-2-2-2-2',
            '4-4-4-4-3-3-3-1-1-1',
            '4-4-4-4-3-3-3-2-1-1',
            '4-4-4-4-3-3-3-2-2-1',
            '4-4-4-4-3-3-3-2-2-2',
            '4-4-4-4-3-3-3-3-1-1',
            '4-4-4-4-3-3-3-3-2-1',
            '4-4-4-4-3-3-3-3-2-2',
            '4-4-4-4-3-3-3-3-3-1',
            '4-4-4-4-3-3-3-3-3-2',
            '4-4-4-4-4-1-1-1-1-1',
            '4-4-4-4-4-2-1-1-1-1',
            '4-4-4-4-4-2-2-1-1-1',
            '4-4-4-4-4-2-2-2-1-1',
            '4-4-4-4-4-2-2-2-2-1',
            '4-4-4-4-4-2-2-2-2-2',
            '4-4-4-4-4-3-1-1-1-1',
            '4-4-4-4-4-3-2-1-1-1',
            '4-4-4-4-4-3-2-2-1-1',
            '4-4-4-4-4-3-2-2-2-1',
            '4-4-4-4-4-3-2-2-2-2',
            '4-4-4-4-4-3-3-1-1-1',
            '4-4-4-4-4-3-3-2-1-1',
            '4-4-4-4-4-3-3-2-2-1',
            '4-4-4-4-4-3-3-2-2-2',
            '4-4-4-4-4-3-3-3-1-1',
            '4-4-4-4-4-3-3-3-2-1',
            '4-4-4-4-4-3-3-3-2-2',
            '4-4-4-4-4-3-3-3-3-1',
            '4-4-4-4-4-3-3-3-3-2',
            '4-4-4-4-4-4-1-1-1-1',
            '4-4-4-4-4-4-2-1-1-1',
            '4-4-4-4-4-4-2-2-1-1',
            '4-4-4-4-4-4-2-2-2-1',
            '4-4-4-4-4-4-2-2-2-2',
            '4-4-4-4-4-4-3-1-1-1',
            '4-4-4-4-4-4-3-2-1-1',
            '4-4-4-4-4-4-3-2-2-1',
            '4-4-4-4-4-4-3-2-2-2',
            '4-4-4-4-4-4-3-3-1-1',
            '4-4-4-4-4-4-3-3-2-1',
            '4-4-4-4-4-4-3-3-2-2',
            '4-4-4-4-4-4-3-3-3-1',
            '4-4-4-4-4-4-3-3-3-2',
            '4-4-4-4-4-4-4-1-1-1',
            '4-4-4-4-4-4-4-2-1-1',
            '4-4-4-4-4-4-4-2-2-1',
            '4-4-4-4-4-4-4-2-2-2',
            '4-4-4-4-4-4-4-3-1-1',
            '4-4-4-4-4-4-4-3-2-1',
            '4-4-4-4-4-4-4-3-2-2',
            '4-4-4-4-4-4-4-3-3-1',
            '4-4-4-4-4-4-4-3-3-2',
            '4-4-4-4-4-4-4-4-1-1',
            '4-4-4-4-4-4-4-4-2-1',
            '4-4-4-4-4-4-4-4-2-2',
            '4-4-4-4-4-4-4-4-3-1',
            '4-4-4-4-4-4-4-4-3-2',
            '4-4-4-4-4-4-4-4-4-1',
            '4-4-4-4-4-4-4-4-4-2']
fastcore.test.test_eq(sorted(profiles), expected)


## Profile filters

The profile filters work out which strategy profiles are relevant to the game in question. I already pass in two filters by default which keep only the strategy profiles which are relevant to the transition the algorithm is curently calculating the fixation probability for, and are possible given the `allowed_sectors` for the players. 

Additional filters could be defined and passed in when desired (e.g. a filter to keep only strategy profiles which are unique to rearrangement)

Technical note: A similar effect could be achieved by having a sampling rule which returned 0 as the likelihood for irrelevant or impossible strategy profiles. However, it usually makes sense to keep these restrictions separate: profile filters can be reused for many possible games, but sampling rules may vary more often. There is also a slight performance advantage to having a profile filter, since it reduces the number of times the sampling rule must be called.

##### Profile filter methods

In [ ]:
#| export

@multi
def profile_filter(models):
    "Filter strategy profiles to those which satisfy the given rule."
    return models.get('profile_filter_rule')

@method(profile_filter, 'allowed_sectors')
def profile_filter(models):
    """Filter strategy profiles to only those where players are from their
    allowed sectors."""
    profiles = models.get('profiles_filtered',
                          models.get('profiles',
                                     create_profiles(models)['profiles']))
    allowed_sectors = models['allowed_sectors']
    sector_strategies = models['sector_strategies']
    profiles_filtered = []
    for k in profiles:
        k_tuple = list(map(int, k.split("-")))
        valid = True
        for i, ind in enumerate(k_tuple[::-1]):
            allowed_inds = np.hstack([sector_strategies[j]
                                      for j in allowed_sectors[f"P{i+1}"]])
            if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
                valid = False
        if valid==True:
            profiles_filtered.append(k)
    return {**models, "profiles_filtered": profiles_filtered}

@method(profile_filter)
def profile_filter(models):
    """The default filter method leaves models unchanged."""
    print("""`profile_filter` called but `models` did not specify a
           `profile_filter_rule`. Try specifying one.""")
    return models

We also need a filter which yields the relevant profiles for each transition considered.

Given two states and their transition we first check that it is valid, and if we so we know
the sector affected. Only that sector may choose different strategies, so the others are fixed.

Note that the recurrent states of the game describes each strategy employed by each sector. This is written in the same form as we write the strategy profile, "{strategy_code}-{strategy_code}-{strategy_code}" and uses the same codes.

**Note:** Below, my filter only keeps strategies which are relevant to the two recurrent states relevant to the transition. If we need additional strategies, this filter will not be sufficient (this might be the case if we allow a social learning rule which explores more than one mutant strategy at a time)

In [ ]:
#| export
@method(profile_filter, 'relevant_to_transition')
def profile_filter(models):
    """Filter for strategy profiles relevant to the given transition."""
    ind1, ind2 = models.get('transition_indices', [None, None])
    if (ind1==None) and (ind2==None):
        return models
    sector_strategies = models['sector_strategies']
    profiles = models.get('profiles_filtered',
                          models.get('profiles',
                                     create_profiles(models)['profiles']))
    strategies1 = list(map(int, ind1.split("-")))
    strategies2 = list(map(int, ind2.split("-")))
    differ = [i1!=i2 for i1, i2 in zip(strategies1, strategies2)]
    # Check states only differ for one sector
    valid = sum(differ) == 1
    # Check that states use valid stratgy codes for each sector
    # Unlike when the states differ by more than one sector, this will only
    # happen if the transition_indices and sctor_strategies are inconsistent,
    # so we raise a value error.
    for i, sector in enumerate(sorted(sector_strategies)):
        if ((strategies1[-(i+1)] not in sector_strategies[sector]
             and str(strategies1[-(i+1)]) not in sector_strategies[sector])
            or (strategies2[-(i+1)] not in sector_strategies[sector]
                and str(strategies2[-(i+1)]) not in sector_strategies[sector])):
            valid = False
            raise ValueError("States use invalid strategy codes for some sectors.")
    if valid:
        strategies_valid = np.unique(np.hstack([strategies1, strategies2]))
        profiles_filtered = []
        for profile in profiles:
            relevant = True
            for strategy in list(map(int, profile.split("-"))):
                if strategy not in strategies_valid:
                    relevant = False
            if relevant == True:
                profiles_filtered.append(profile)
        return {**models, "profiles_filtered": profiles_filtered}
    return models

I have also written a function for applying multiple filters.

In [ ]:
#| export
def apply_profile_filters(models):
    "Apply all profile filters listed in `profile_filters` in `models`."
    for rule in models.get('profile_filters', 
                           ["allowed_sectors",
                            "relevant_to_transition"]):
        models = profile_filter({**models, "profile_filter_rule": rule})
    return models

##### Tests for `profile_filter`

##### Test 1

Let's test the `"allowed_sectors"` filter rule.

First I test a game with 3 players but only two sectors.\
Each player is fixed to a specific sector, but two players belong to the same sector.\
In this case the rule should filter to only those profiles where players use\
the strategies available to the sectors they can play as.

In [ ]:
#| export
allowed_sectors = {"P3": ["S2"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S2": [2, 3],
                     "S1": [0, 1]}
n_players = 3
n_strategies = [2, 2] # this could be derived from sector_strategies or the other way round.
models = {"profile_filter_rule": "allowed_sectors",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies}
result = profile_filter(models)['profiles_filtered']
# Only strategy 2 is irrelevant
expected = ["2-2-0", "2-2-1", 
            "2-3-0", "2-3-1",
            "3-2-0", "3-2-1",
            "3-3-0", "3-3-1",]
fastcore.test.test_eq(result, expected)
fastcore.test.test_eq(len(result), 8)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


I then test a 3 player game with 3 sectors. Each player is fixed to a particular sector.

In [ ]:
#| export
allowed_sectors = {"P3": ["S3"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S3": [4, 5],
                     "S2": [2, 3],
                     "S1": [0, 1]}
models = {"profile_filter_rule": "allowed_sectors",
          "n_players": 3,
          "n_strategies": [2, 2, 2],
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies}
result = profile_filter(models)['profiles_filtered']
expected = ["4-2-0",
            "4-2-1",
            "4-3-0",
            "4-3-1",
            "5-2-0",
            "5-2-1",
            "5-3-0",
            "5-3-1",
           ]
fastcore.test.test_eq(result, expected)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


##### Test 2

Now, let's test the `"relevant_to_transition"` filter rule.

First I test a game with 3 players but only two populations.\
In this case the rule should filter to only the relevant profiles where strategy 2 is missing.\
Recall that strategy 2 is the first strategy available to a player from sector 2.

In [ ]:
#| export
sector_strategies = {"S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["3-0", "3-1"]
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1"],
                   "P3": ["S2"],}
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result = profile_filter(models)['profiles_filtered']
# Only strategy 2 is irrelevant
expected = ["0-0-0", "0-0-1", "0-0-3", 
            "0-1-0", "0-1-1", "0-1-3",
            "0-3-0", "0-3-1", "0-3-3",
            "1-0-0", "1-0-1", "1-0-3",
            "1-1-0", "1-1-1", "1-1-3",
            "1-3-0", "1-3-1", "1-3-3",
            "3-0-0", "3-0-1", "3-0-3",
            "3-1-0", "3-1-1", "3-1-3",
            "3-3-0", "3-3-1", "3-3-3"]
fastcore.test.test_eq(result, expected)
expected = (np.sum(n_strategies) - 1) ** n_players  # 1 of the 4 strategies won't be relevant here
fastcore.test.test_eq(len(result), expected)

I then test a larger game with 3 players and 3 sectors. The list is long, so
I only check that the number of profiles kept is what we expect.

In [ ]:
#| export
sector_strategies = {"S3": [4, 5],
                     "S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["5-3-1", "5-3-0"]
n_players = 3
n_strategies = [2, 2, 2]
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result = profile_filter(models)['profiles_filtered']
expected = (np.sum(n_strategies) - 2) ** n_players  # 2 of the 6 strategies won't be relevant here
fastcore.test.test_eq(len(result), expected)

##### Test 3

I now test the `"allowed_sectors"` and "`relevant_to_transition`" rules when used together.\
The game is as before with 3 players and 3 sectors. Here, we can check that the list of profiles is as expected.

In [ ]:
#| export
allowed_sectors = {"P3": ["S3"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S3": [4, 5],
                     "S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["5-3-1", "5-3-0"]
n_players = 3
n_strategies = [2, 2, 2]
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result = thread_macro(models,
                      profile_filter,
                      (assoc, "profile_filter_rule", "allowed_sectors"),
                      profile_filter)
expected = ["5-3-0", "5-3-1"]
fastcore.test.test_eq(len(result['profiles_filtered']), 2)
fastcore.test.test_eq(result['profiles_filtered'], expected)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


The order we apply these two filter rules should not matter.

In [ ]:
#| export
allowed_sectors = {"P3": ["S3"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S3": [4, 5],
                     "S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["5-3-1", "5-3-0"]
n_players = 3
n_strategies = [2, 2, 2]
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result1 = thread_macro(models,
                       profile_filter,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter)
result2 = thread_macro(models,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter,
                       (assoc, "profile_filter_rule", "relevant_to_transition"),
                       profile_filter)
expected = ["5-3-0", "5-3-1"]
fastcore.test.test_eq(result1['profiles_filtered'], expected)
fastcore.test.test_eq(result2['profiles_filtered'], expected)
fastcore.test.test_eq(result1['profiles_filtered'],
                      result2['profiles_filtered'])

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


We can use the `apply_profile_filters` function to achieve the same result.

In [ ]:
#| export
allowed_sectors = {"P3": ["S3"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S3": [4, 5],
                     "S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["5-3-1", "5-3-0"]
n_players = 3
n_strategies = [2, 2, 2]
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
models = assoc(models, 
               'profile_filters', 
               ["allowed_sectors", "relevant_to_transition"])
result = apply_profile_filters(models)
expected = ["5-3-0", "5-3-1"]
fastcore.test.test_eq(result['profiles_filtered'], expected)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


Let's also check the eariler game with 2 sectors and 3 players.

In [ ]:
#| export
allowed_sectors = {"P3": ["S2"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["3-0", "3-1"]
n_players = 3
n_strategies = [2, 2] # this could be derived from sector_strategies or the other way round.
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result = apply_profile_filters(models)
result1 = thread_macro(models,
                       profile_filter,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter)
result2 = thread_macro(models,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter,
                       (assoc, "profile_filter_rule", "relevant_to_transition"),
                       profile_filter)
expected = ["3-3-0", "3-3-1"]
fastcore.test.test_eq(result1['profiles_filtered'], expected)
fastcore.test.test_eq(result2['profiles_filtered'], expected)
fastcore.test.test_eq(result1['profiles_filtered'],
                      result2['profiles_filtered'])

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


Notice that in this game, transitions which affect the sector with multiple players require us to look at more profiles.

In [ ]:
#| export
allowed_sectors = {"P3": ["S2"],
                   "P2": ["S2"],
                   "P1": ["S1"]}
sector_strategies = {"S2": [2, 3],
                     "S1": [0, 1]}
transition_indices = ["2-1", "3-1"]
n_players = 3
n_strategies = [2, 2] # this could be derived from sector_strategies or the other way round.
models = {"profile_filter_rule": "relevant_to_transition",
          "n_players": n_players,
          "n_strategies": n_strategies,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "transition_indices": transition_indices}
result1 = thread_macro(models,
                       profile_filter,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter)
result2 = thread_macro(models,
                       (assoc, "profile_filter_rule", "allowed_sectors"),
                       profile_filter,
                       (assoc, "profile_filter_rule", "relevant_to_transition"),
                       profile_filter)
models = assoc(models, 
               'profile_filters', 
               ["allowed_sectors", "relevant_to_transition"])
result3 = apply_profile_filters(models)
expected = ["2-2-1", "2-3-1", "3-2-1", "3-3-1"]
fastcore.test.test_eq(result1['profiles_filtered'], expected)
fastcore.test.test_eq(result2['profiles_filtered'], expected)
fastcore.test.test_eq(result3['profiles_filtered'], expected)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


### An additional method for `create_profiles` which depends on `profile_filter`

In [ ]:
#| export
@method(create_profiles, "allowed_sectors")
def create_profiles(models):
    """Create all strategy profiles for the set of models."""
    profiles = thread_macro(create_profiles({**models, "profiles_rule": None}),
                            (assoc, "profile_filter_rule", "allowed_sectors"),
                            profile_filter,
                            (get, "profiles_filtered"),
                            )
    return {**models, "profiles": profiles}


Here is a quick test of the "allowed_sectors" method

In [ ]:
allowed_sectors = {"P1": ["S1"],
                   "P2": ["S1", "S2"], }
# Third sector irrelevant given allowed sectors
sector_strategies = {"S1": ["1", "2"],
                     "S2": ["3", "4"],
                     "S3": ["5", "6"]}
models = {"allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "profiles_rule": "allowed_sectors"}
profiles = create_profiles(models)['profiles']
expected = ['1-1', "1-2", '2-1', '2-2', '3-1', '3-2', '4-1', '4-2']
fastcore.test.test_eq(sorted(profiles), expected)

/tmp/ipykernel_4122/3464752654.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()